# Goal:
1. The task is to forecast the sales for every item in every shop in the testing period.
2. To apply different strategies and learn.

In [1]:
# Importing required libraries
import pandas as pd

# Visualization libraries
import matplotlib.pyplot as plt
import seaborn as sns

import datetime as dt

In [2]:
# Import customized py library
import customized_py_library as Cus_Lib

In [3]:
# Loading Data
Train_Data = pd.read_csv("../input/competitive-data-science-predict-future-sales/sales_train.csv")
Test_Data = pd.read_csv("../input/competitive-data-science-predict-future-sales/test.csv")
sample_sub = pd.read_csv("../input/competitive-data-science-predict-future-sales/sample_submission.csv")

# Understanding Data

In [4]:
sample_sub.head()

,ID,item_cnt_month
0,0,0.5
1,1,0.5
2,2,0.5
3,3,0.5
4,4,0.5


In [5]:
# Understanding Train Data
# Shape of Data
Train_Data.shape, Test_Data.shape

# Info of Data
#Train_Data.info(), Test_Data.info()

# Print the Data
#Train_Data.head()
#Test_Data.head()

((2935849, 6), (214200, 3))

**Exploring the Train Dataset**

In [6]:
# Unique no. of IDs
Train_Data['shop_id'].nunique(), Train_Data['item_id'].nunique()

(60, 21807)

In [7]:
# Datetime setting
Cus_Lib.Datetimeset(Train_Data)

,date_block_num,shop_id,item_id,item_price,item_cnt_day,day,month,year
49800,0,18,5823,2500.0,1.0,1,1,2013
29784,0,27,5573,849.0,1.0,1,1,2013
35476,0,7,1006,399.0,1.0,1,1,2013
8330,0,19,17707,899.0,1.0,1,1,2013
57384,0,14,19548,149.0,1.0,1,1,2013
...,...,...,...,...,...,...,...,...
2885098,33,41,21386,169.0,1.0,31,10,2015
2930981,33,21,988,199.0,1.0,31,10,2015
2885097,33,41,21377,169.0,1.0,31,10,2015
2930993,33,22,10207,1199.0,1.0,31,10,2015


**Finding percentage of Datasets before merging**

In [8]:
Train_Data.head()

,date_block_num,shop_id,item_id,item_price,item_cnt_day,day,month,year
49800,0,18,5823,2500.0,1.0,1,1,2013
29784,0,27,5573,849.0,1.0,1,1,2013
35476,0,7,1006,399.0,1.0,1,1,2013
8330,0,19,17707,899.0,1.0,1,1,2013
57384,0,14,19548,149.0,1.0,1,1,2013


In [9]:
Cus_Lib.Percent(2935849, 214200)

No. of Observations (Training Dataset): 0.9320010577613237
No. of Observations (Testing Dataset): 0.0679989422386763


**Lets Merge Training and Testing Dataset for EDA**

In [10]:
#Merged_Data = pd.merge(Train_Data, Test_Data, how = 'left')
DF = pd.merge(Test_Data, Train_Data, on = ('shop_id', 'item_id'), how = 'left')

In [11]:
DF.shape

(1327235, 9)

In [12]:
Cus_Lib.Drop_duplicate_value(DF)

Duplcated observations: 5
Droped observations: 5
New Shape: (1327230, 9)


In [13]:
DF['shop_id'].nunique(), DF['item_id'].nunique()

(42, 5100)

# Exploratory Data Analysis

1. Data Cleaning.
2. Data Preprocessing.
3. Data visualization.

1. Data Cleaning

In [14]:
Cus_Lib.find_NaN_V(DF)

,NaN values,NaN percent
ID,0,0.000000
shop_id,0,0.000000
item_id,0,0.000000
date_block_num,102796,7.745124
item_price,102796,7.745124
item_cnt_day,102796,7.745124
day,102796,7.745124
month,102796,7.745124
year,102796,7.745124


# Train & Test split